In [1]:
!pip install ..
!pip install rouge-score

Processing /mnt/HQ/uni/text-mining/multidocsum
  Created wheel for multidocsum: filename=multidocsum-0.0.1-py3-none-any.whl size=8467 sha256=ac80a858acda8fc054afc945a174915217a3a18ed74f3638002765ede6d827c4
  Stored in directory: /tmp/pip-ephem-wheel-cache-o0mr3yf4/wheels/02/74/95/c7b282f469bd75d8476184f75ec79b255b390a90201a66b918
Successfully built multidocsum
  Attempting uninstall: multidocsum
    Found existing installation: multidocsum 0.0.1
    Uninstalling multidocsum-0.0.1:
      Successfully uninstalled multidocsum-0.0.1


In [2]:
from src.models.extracting import MEAD
from src.data.data_io import DataIO
import numpy as np
from glob import glob
import numpy as np

In [3]:
cluster_name = []
for path in glob('../data/interim/extract_content/*'):
    cluster_name.append(path.split('/')[-1])

In [4]:
%%time
my_sum = {}
gold_sum = {}
for name in cluster_name:
    params = {
        'data_path': f'../data/interim/extract_content/{name}',
        'stop_words_path': '../data/external/stopwords.txt', 
        'punc_path': '../data/external/punctuation.txt',
        'compression_rate': 0.05,
        'remove_redundancy': True,
        'wc': 1.0, 
        'wp': 1.0, 
        'wf': 1.0, 
        'n_centroid': 10, 
        'C_max': 10, 
        'ngram_range': (1, 1), 
        'max_features': 100,
        'func': np.dot
    }
    summary = MEAD.extract(**params)
    os.makedirs(f'../summaries/{name}', exist_ok=True)
    with open(f'../summaries/{name}/summary.txt', 'w') as f:
        f.write('\n'.join(summary))
        my_sum[name] = '\n'.join(summary)
        gold_sum[name] = DataIO.load(f'../data/raw/ViMs/summary/{name}')

CPU times: user 26min 23s, sys: 795 ms, total: 26min 24s
Wall time: 26min 41s


In [5]:
from rouge_score import rouge_scorer

In [6]:
rougen = ['rouge1', 'rouge2', 'rouge3', 'rouge4']
scorer = rouge_scorer.RougeScorer(rougen)

In [7]:
%%time
mean_0 = {}
for n in rougen:
    mean_0[n] = {}
    rn = mean_0[n]
    precision = []
    recall = []
    fmeasure = []
    for cluster in my_sum:
        precision.append(scorer.score(gold_sum[cluster]['0.gold.txt'], my_sum[cluster])[n].precision)
        recall.append(scorer.score(gold_sum[cluster]['0.gold.txt'], my_sum[cluster])[n].recall)
        fmeasure.append(scorer.score(gold_sum[cluster]['0.gold.txt'], my_sum[cluster])[n].fmeasure)
    rn['precision'] = np.mean(precision)
    rn['recall'] = np.mean(recall)
    rn['f-measure'] = np.mean(fmeasure)

CPU times: user 12.5 s, sys: 0 ns, total: 12.5 s
Wall time: 12.6 s


In [8]:
import pandas as pd
pd.DataFrame(mean_0)

,rouge1,rouge2,rouge3,rouge4
precision,0.709228,0.445738,0.304384,0.252520
recall,0.724331,0.457173,0.311050,0.257734
f-measure,0.670701,0.421880,0.288323,0.239300


In [9]:

mean_1 = {}
for n in rougen:
    mean_1[n] = {}
    rn = mean_1[n]
    precision = []
    recall = []
    fmeasure = []
    for cluster in my_sum:
        precision.append(scorer.score(gold_sum[cluster]['1.gold.txt'], my_sum[cluster])[n].precision)
        recall.append(scorer.score(gold_sum[cluster]['1.gold.txt'], my_sum[cluster])[n].recall)
        fmeasure.append(scorer.score(gold_sum[cluster]['1.gold.txt'], my_sum[cluster])[n].fmeasure)
    rn['precision'] = np.mean(precision)
    rn['recall'] = np.mean(recall)
    rn['f-measure'] = np.mean(fmeasure)

In [10]:
pd.DataFrame(mean_1)

,rouge1,rouge2,rouge3,rouge4
precision,0.762609,0.490683,0.346229,0.291536
recall,0.704133,0.453261,0.318807,0.268423
f-measure,0.696245,0.447542,0.315653,0.265948
